In [3]:
import pandas as pd
import h5py
test_case = '/ruiyan/yuhao/tile_embed/BRACS/UNI/BRACS_1926.h5'
with h5py.File(test_case, 'r') as f:
    coords = f['coords'][:]
    feature = f['features'][:]


In [4]:
feature.shape

(38717, 1024)

: 

In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: wyh196646. Use `wandb login --relogin` to force relogin


acc,▁▄▅▇▇███
loss,█▅▃▁▁▁▁▁
acc,0.8443
loss,0.14803


In [6]:
import h5py
def read_assets_from_h5( h5_path: str) -> tuple:
    '''Read the assets from the h5 file'''
    assets = {}
    attrs = {}
    with h5py.File(h5_path, 'r') as f:
        for key in f.keys():
            assets[key] = f[key][:]
            if f[key].attrs is not None:
                attrs[key] = dict(f[key].attrs)
    return assets, attrs
assets,_ = read_assets_from_h5('/ruiyan/yuhao/tile_embed/BRACS/BRACS/CONCH/BRACS_1231.h5')

In [7]:
assets['features'].shape

(80279, 768)

In [8]:
assets['features'].shape

(80279, 768)